In [1]:
import cv2
import numpy as np
import os
from glob import glob
from scipy.signal import savgol_filter
import shutil

In [2]:
def load_frames_from_directory(frames_path):
    """Load frames from directory in sorted order"""
    frame_files = []
    for filename in sorted(os.listdir(frames_path)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            frame_files.append(os.path.join(frames_path, filename))
    return frame_files

def detect_and_match_features_fast(img1, img2):
    # FAST keypoints + ORB descriptors for speed and robustness
    fast = cv2.FastFeatureDetector_create(threshold=60)
    orb = cv2.ORB_create(nfeatures=1000)
    kp1 = fast.detect(img1, None)
    kp2 = fast.detect(img2, None)
    kp1, des1 = orb.compute(img1, kp1)
    kp2, des2 = orb.compute(img2, kp2)
    if des1 is None or des2 is None:
        return np.array([]), np.array([])
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    if len(matches) < 4:
        return np.array([]), np.array([])
    good_matches = matches[:min(50, len(matches))]
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches])
    return pts1, pts2

def filter_motion_outliers(pts1, pts2):
    motion_vectors = pts2 - pts1
    motion_magnitudes = np.linalg.norm(motion_vectors, axis=1)
    median_motion = np.median(motion_magnitudes)
    mad = np.median(np.abs(motion_magnitudes - median_motion))
    threshold = median_motion + 2 * mad
    static_mask = motion_magnitudes < threshold
    return pts1[static_mask], pts2[static_mask]

def create_water_mask(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_water = np.array([90, 0, 50])
    upper_water = np.array([130, 100, 200])
    water_mask = cv2.inRange(hsv, lower_water, upper_water)
    kernel = np.ones((5,5), np.uint8)
    water_mask = cv2.morphologyEx(water_mask, cv2.MORPH_CLOSE, kernel)
    return water_mask

def filter_features_by_region(pts1, pts2, frame):
    water_mask = create_water_mask(frame)
    valid_mask = []
    for pt in pts1:
        x, y = int(pt[0]), int(pt[1])
        if 0 <= x < water_mask.shape[1] and 0 <= y < water_mask.shape[0]:
            valid_mask.append(water_mask[y, x] == 0)
        else:
            valid_mask.append(True)
    valid_mask = np.array(valid_mask)
    return pts1[valid_mask], pts2[valid_mask]

def estimate_homography(pts1, pts2):
    if len(pts1) >= 4 and len(pts2) >= 4:
        H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)
        if H is None:
            return np.eye(3)
        return H
    else:
        return np.eye(3)

def adaptive_trajectory_smoothing(trajectory, motion_threshold=10, pass_number=1):
    """Enhanced smoothing that gets more aggressive with each pass"""
    trajectory = np.array(trajectory)
    smoothed = np.zeros_like(trajectory)
    
    # Make smoothing more aggressive for later passes
    base_window = 15 if pass_number == 1 else 25
    aggressive_window = 5 if pass_number == 1 else 7
    
    for i in range(trajectory.shape[1]):
        component = trajectory[:, i]
        motion_magnitude = np.abs(np.diff(component))
        
        if np.any(motion_magnitude > motion_threshold):
            window_length = min(aggressive_window, len(component))
            if window_length % 2 == 0:
                window_length += 1
            smoothed[:, i] = savgol_filter(component, window_length=window_length, polyorder=2)
        else:
            window_length = min(base_window, len(component))
            if window_length % 2 == 0:
                window_length += 1
            smoothed[:, i] = savgol_filter(component, window_length=window_length, polyorder=3)
    return smoothed

def extract_frames_from_video(video_path, output_frames_dir):
    """Extract frames from video to directory for next pass"""
    if os.path.exists(output_frames_dir):
        shutil.rmtree(output_frames_dir)
    os.makedirs(output_frames_dir)
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_filename = os.path.join(output_frames_dir, f"frame_{frame_count:06d}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_count += 1
    
    cap.release()
    print(f"Extracted {frame_count} frames to {output_frames_dir}")
    return frame_count

def improved_stabilization_from_frames(frames_path, output_path, fps=29.97, pass_number=1):
    """
    Stabilization using pre-extracted frames from directory
    """
    # Load frame file paths
    frame_files = load_frames_from_directory(frames_path)
    total_frames = len(frame_files)
    
    if total_frames == 0:
        print("Error: No frames found in directory")
        return False
    
    print(f"Pass {pass_number}: Found {total_frames} frames in directory")
    
    # Read first frame to get dimensions
    first_frame = cv2.imread(frame_files[0])
    if first_frame is None:
        print("Error: Could not read first frame")
        return False
    
    h, w = first_frame.shape[:2]
    
    # Setup output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))
    
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    transforms = []
    trajectory = [np.eye(3)]
    
    print(f"Pass {pass_number} - Phase 1: Computing transforms...")
    
    # Process frames from directory
    for i in range(1, total_frames):
        curr_frame = cv2.imread(frame_files[i])
        if curr_frame is None:
            print(f"Warning: Could not read frame {i}")
            transforms.append(np.eye(3))
            trajectory.append(trajectory[-1])
            continue
        
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        pts1, pts2 = detect_and_match_features_fast(prev_gray, curr_gray)
        
        if len(pts1) > 0:
            pts1, pts2 = filter_motion_outliers(pts1, pts2)
        if len(pts1) > 0:
            pts1, pts2 = filter_features_by_region(pts1, pts2, curr_frame)
        
        H = estimate_homography(pts1, pts2)
        transforms.append(H)
        trajectory.append(trajectory[-1] @ H)
        prev_gray = curr_gray
        
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{total_frames} frames")
    
    print(f"Pass {pass_number} - Phase 2: Smoothing trajectory...")
    
    # Extract parameters and smooth
    params = []
    for H in trajectory:
        dx = H[0, 2]
        dy = H[1, 2]
        da = np.arctan2(H[1, 0], H[0, 0])
        params.append([dx, dy, da])
    
    # Apply more aggressive smoothing for later passes
    motion_threshold = 15 - (pass_number - 1) * 3  # More aggressive each pass
    smoothed_params = adaptive_trajectory_smoothing(params, motion_threshold=motion_threshold, pass_number=pass_number)
    
    # Calculate smoothed transforms
    new_transforms = []
    for i in range(len(smoothed_params)):
        dx, dy, da = smoothed_params[i]
        cos_a = np.cos(da)
        sin_a = np.sin(da)
        new_H = np.array([[cos_a, -sin_a, dx],
                          [sin_a,  cos_a, dy],
                          [0,      0,     1]], dtype=np.float32)
        new_transforms.append(new_H)
    
    print(f"Pass {pass_number} - Phase 3: Applying stabilization and writing video...")
    
    # Apply stabilization and write video
    for i in range(total_frames):
        frame = cv2.imread(frame_files[i])
        if frame is None:
            continue
        
        if i < len(new_transforms):
            H = new_transforms[i]
            if H is not None and not np.isnan(H).any() and not np.isinf(H).any():
                try:
                    frame_stabilized = cv2.warpPerspective(frame, H, (w, h), borderMode=cv2.BORDER_REFLECT)
                except:
                    frame_stabilized = frame
            else:
                frame_stabilized = frame
        else:
            frame_stabilized = frame
        
        out.write(frame_stabilized)
        
        if (i + 1) % 100 == 0:
            print(f"Stabilized and wrote {i + 1}/{total_frames} frames")
    
    out.release()
    print(f"Pass {pass_number} completed: Stabilized video saved at: {output_path}")
    return True

def multi_pass_stabilization(frames_path, final_output_path, num_passes=3, fps=29.97):
    """
    Multi-pass stabilization for achieving 85-90% motion reduction
    """
    print(f"Starting multi-pass stabilization with {num_passes} passes...")
    print(f"Expected motion reduction: 75-90%")
    
    temp_dirs = []
    temp_videos = []
    
    try:
        for pass_num in range(num_passes):
            print(f"\n{'='*50}")
            print(f"STARTING PASS {pass_num + 1}/{num_passes}")
            print(f"{'='*50}")
            
            # Determine input and output paths
            if pass_num == 0:
                # First pass: use original frames
                input_frames = frames_path
            else:
                # Subsequent passes: extract frames from previous pass video
                temp_frames_dir = f"temp_frames_pass_{pass_num}"
                temp_dirs.append(temp_frames_dir)
                extract_frames_from_video(temp_videos[-1], temp_frames_dir)
                input_frames = temp_frames_dir
            
            # Output path
            if pass_num == num_passes - 1:
                # Final pass: use final output path
                output_video = final_output_path
            else:
                # Intermediate pass: use temporary video
                output_video = f"temp_stabilized_pass_{pass_num + 1}.mp4"
                temp_videos.append(output_video)
            
            # Run stabilization pass
            success = improved_stabilization_from_frames(
                input_frames, 
                output_video, 
                fps=fps, 
                pass_number=pass_num + 1
            )
            
            if not success:
                print(f"Pass {pass_num + 1} failed!")
                break
                
            print(f"Pass {pass_num + 1} completed successfully!")
        
        print(f"\n{'='*50}")
        print(f"MULTI-PASS STABILIZATION COMPLETED!")
        print(f"Final stabilized video: {final_output_path}")
        print(f"Expected motion reduction: 75-90%")
        print(f"{'='*50}")
        
    finally:
        # Cleanup temporary files
        print("\nCleaning up temporary files...")
        for temp_dir in temp_dirs:
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir)
                print(f"Removed temporary directory: {temp_dir}")
        
        for temp_video in temp_videos:
            if os.path.exists(temp_video):
                os.remove(temp_video)
                print(f"Removed temporary video: {temp_video}")
        
        print("Cleanup completed!")

# USAGE: Multi-pass stabilization for maximum motion reduction
frames_path = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\unzipped_video"
final_output_path = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\MultiPass_UltraStable.mp4"

# Run multi-pass stabilization (3 passes for 85-90% motion reduction)
multi_pass_stabilization(
    frames_path=frames_path,
    final_output_path=final_output_path,
    num_passes=3,  # Adjust: 2 passes = ~75-80%, 3 passes = ~85-90%
    fps=29.97
)

Starting multi-pass stabilization with 3 passes...
Expected motion reduction: 75-90%

STARTING PASS 1/3
Pass 1: Found 7169 frames in directory
Pass 1 - Phase 1: Computing transforms...
Processed 100/7169 frames
Processed 200/7169 frames
Processed 300/7169 frames
Processed 400/7169 frames
Processed 500/7169 frames
Processed 600/7169 frames
Processed 700/7169 frames
Processed 800/7169 frames
Processed 900/7169 frames
Processed 1000/7169 frames
Processed 1100/7169 frames
Processed 1200/7169 frames
Processed 1300/7169 frames
Processed 1400/7169 frames
Processed 1500/7169 frames
Processed 1600/7169 frames
Processed 1700/7169 frames
Processed 1800/7169 frames
Processed 1900/7169 frames
Processed 2000/7169 frames
Processed 2100/7169 frames
Processed 2200/7169 frames
Processed 2300/7169 frames
Processed 2400/7169 frames
Processed 2500/7169 frames
Processed 2600/7169 frames
Processed 2700/7169 frames
Processed 2800/7169 frames
Processed 2900/7169 frames
Processed 3000/7169 frames
Processed 3100